In [12]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import tensorflow as tf

# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Import our input dataset
loans_df = pd.read_csv('loan_status.csv')
loans_df.head()

,Loan_Status,Current_Loan_Amount,Term,Credit_Score,Annual_Income,Years_in_current_job,Home_Ownership,Purpose,Monthly_Debt,Years_of_Credit_History,Months_since_last_delinquent,Number_of_Open_Accounts,Number_of_Credit_Problems,Current_Credit_Balance,Maximum_Open_Credit,Bankruptcies,Tax_Liens
0,Fully_Paid,99999999,Short_Term,741,2231892,8_years,Own_Home,Debt_Consolidation,29200.53,14.9,29,18,1,297996,750090,0,0
1,Fully_Paid,217646,Short_Term,730,1184194,<_1_year,Home_Mortgage,Debt_Consolidation,10855.08,19.6,10,13,1,122170,272052,1,0
2,Fully_Paid,548746,Short_Term,678,2559110,2_years,Rent,Debt_Consolidation,18660.28,22.6,33,4,0,437171,555038,0,0
3,Fully_Paid,99999999,Short_Term,728,714628,3_years,Rent,Debt_Consolidation,11851.06,16.0,76,16,0,203965,289784,0,0
4,Fully_Paid,99999999,Short_Term,740,776188,<_1_year,Own_Home,Debt_Consolidation,11578.22,8.5,25,6,0,134083,220220,0,0


In [2]:
loans_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36423 entries, 0 to 36422
Data columns (total 17 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Loan_Status                   36423 non-null  object 
 1   Current_Loan_Amount           36423 non-null  int64  
 2   Term                          36423 non-null  object 
 3   Credit_Score                  36423 non-null  int64  
 4   Annual_Income                 36423 non-null  int64  
 5   Years_in_current_job          36423 non-null  object 
 6   Home_Ownership                36423 non-null  object 
 7   Purpose                       36423 non-null  object 
 8   Monthly_Debt                  36423 non-null  float64
 9   Years_of_Credit_History       36423 non-null  float64
 10  Months_since_last_delinquent  36423 non-null  int64  
 11  Number_of_Open_Accounts       36423 non-null  int64  
 12  Number_of_Credit_Problems     36423 non-null  int64  
 13  C

In [3]:
# Generate our categorical variable list
loans_cat = loans_df.dtypes[loans_df.dtypes == "object"].index.tolist()

# Check the number of unique values in each column
loans_df[loans_cat].nunique()

Loan_Status              2
Term                     2
Years_in_current_job    11
Home_Ownership           4
Purpose                  7
dtype: int64

In [4]:
# Check the unique value counts to see if binning is required
loans_df.Years_in_current_job.value_counts()

10+_years    13149
2_years       3225
3_years       2997
<_1_year      2699
5_years       2487
4_years       2286
1_year        2247
6_years       2109
7_years       2082
8_years       1675
9_years       1467
Name: Years_in_current_job, dtype: int64

In [5]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(loans_df[loans_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(loans_cat)
encode_df.head()

,Loan_Status_Fully_Paid,Loan_Status_Not_Paid,Term_Long_Term,Term_Short_Term,Years_in_current_job_10+_years,Years_in_current_job_1_year,Years_in_current_job_2_years,Years_in_current_job_3_years,Years_in_current_job_4_years,Years_in_current_job_5_years,...,Home_Ownership_Home_Mortgage,Home_Ownership_Own_Home,Home_Ownership_Rent,Purpose_Business_Loan,Purpose_Buy_House,Purpose_Buy_a_Car,Purpose_Debt_Consolidation,Purpose_Home_Improvements,Purpose_Medical_Bills,Purpose_Other
0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [6]:
# Merge one-hot encoded features and drop the originals
loans_df = loans_df.merge(encode_df,left_index=True, right_index=True)
loans_df = loans_df.drop(loans_cat,1)
loans_df.head()

,Current_Loan_Amount,Credit_Score,Annual_Income,Monthly_Debt,Years_of_Credit_History,Months_since_last_delinquent,Number_of_Open_Accounts,Number_of_Credit_Problems,Current_Credit_Balance,Maximum_Open_Credit,...,Home_Ownership_Home_Mortgage,Home_Ownership_Own_Home,Home_Ownership_Rent,Purpose_Business_Loan,Purpose_Buy_House,Purpose_Buy_a_Car,Purpose_Debt_Consolidation,Purpose_Home_Improvements,Purpose_Medical_Bills,Purpose_Other
0,99999999,741,2231892,29200.53,14.9,29,18,1,297996,750090,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,217646,730,1184194,10855.08,19.6,10,13,1,122170,272052,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,548746,678,2559110,18660.28,22.6,33,4,0,437171,555038,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,99999999,728,714628,11851.06,16.0,76,16,0,203965,289784,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,99999999,740,776188,11578.22,8.5,25,6,0,134083,220220,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [7]:
# Remove loan status target from features data
y = loans_df.Loan_Status_Fully_Paid
X = loans_df.drop(columns=["Loan_Status_Fully_Paid","Loan_Status_Not_Paid"])

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [8]:
#following code to the notebook:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  24
hidden_nodes_layer2 = 12

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Train on 27317 samples
Epoch 1/50
27317/27317 [==============================] - 1s 40us/sample - loss: 0.4233 - accuracy: 0.8355
Epoch 2/50
27317/27317 [==============================] - 1s 24us/sample - loss: 0.3868 - accuracy: 0.8489
Epoch 3/50
27317/27317 [==============================] - 1s 24us/sample - loss: 0.3826 - accuracy: 0.8488
Epoch 4/50
27317/27317 [==============================] - 1s 26us/sample - loss: 0.3808 - accuracy: 0.8495
Epoch 5/50
27317/27317 [==============================] - 1s 25us/sample - loss: 0.3793 - accuracy: 0.8496
Epoch 6/50
27317/27317 [==============================] - 1s 25us/sample - loss: 0.3784 - accuracy: 0.8493
Epoch 7/50
27317/27317 [==============================] - 1s 24us/sample - loss: 0.3779 - accuracy: 0.8496
Epoch 8/50
27317/27317 [==============================] - 1s 25us/sample - loss: 0.3770 - accuracy: 0.8494
Epoch 9/50
27317/27317 [==============================] - 1s 25us/sample - loss: 0.3761 - accuracy: 0.8493
Epoch 10/50
27

In [9]:
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 24)                888       
_________________________________________________________________
dense_1 (Dense)              (None, 12)                300       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 13        
Total params: 1,201
Trainable params: 1,201
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=1000)

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Train on 27317 samples
Epoch 1/100
   32/27317 [..............................] - ETA: 4:15 - loss: 0.3923 - accuracy: 0.8750
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 1248/27317 [>.............................] - ETA: 7s - loss: 0.3672 - accuracy: 0.8526  
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 2912/27317 [==>...........................] - ETA: 3s - loss: 0.3625 - accuracy: 0.8551
Epoch 00001: saving model to checkpoints/weights.01.hdf5

Epoch 00001: saving model to checkpoints/weights.01.hdf5
 4096/27317 [===>..........................] - ETA: 2s - loss: 0.3683 - accuracy: 0.8506
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 5536/27317 [=====>........................] - ETA: 1s - loss: 0.3689 - accuracy: 0.8501
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 6656/27317 [======>.......................] - ETA: 1s - loss: 0.3635 - accuracy: 0.8523
Epoch 00001: saving model to checkpoints/weights.01.hdf5
 7936/27317 [=======>......

Epoch 00003: saving model to checkpoints/weights.03.hdf5
10400/27317 [==========>...................] - ETA: 0s - loss: 0.3616 - accuracy: 0.8526
Epoch 00003: saving model to checkpoints/weights.03.hdf5
11744/27317 [===========>..................] - ETA: 0s - loss: 0.3624 - accuracy: 0.8530
Epoch 00003: saving model to checkpoints/weights.03.hdf5

Epoch 00003: saving model to checkpoints/weights.03.hdf5
12928/27317 [=============>................] - ETA: 0s - loss: 0.3617 - accuracy: 0.8540
Epoch 00003: saving model to checkpoints/weights.03.hdf5
14464/27317 [==============>...............] - ETA: 0s - loss: 0.3629 - accuracy: 0.8536
Epoch 00003: saving model to checkpoints/weights.03.hdf5
15968/27317 [================>.............] - ETA: 0s - loss: 0.3633 - accuracy: 0.8532
Epoch 00003: saving model to checkpoints/weights.03.hdf5

Epoch 00003: saving model to checkpoints/weights.03.hdf5
17024/27317 [=================>............] - ETA: 0s - loss: 0.3634 - accuracy: 0.8532
Epoch 00

20192/27317 [=====================>........] - ETA: 0s - loss: 0.3583 - accuracy: 0.8539
Epoch 00005: saving model to checkpoints/weights.05.hdf5
21728/27317 [======================>.......] - ETA: 0s - loss: 0.3591 - accuracy: 0.8541
Epoch 00005: saving model to checkpoints/weights.05.hdf5

Epoch 00005: saving model to checkpoints/weights.05.hdf5
22848/27317 [========================>.....] - ETA: 0s - loss: 0.3593 - accuracy: 0.8540
Epoch 00005: saving model to checkpoints/weights.05.hdf5
23840/27317 [=========================>....] - ETA: 0s - loss: 0.3595 - accuracy: 0.8538
Epoch 00005: saving model to checkpoints/weights.05.hdf5
25280/27317 [==========================>...] - ETA: 0s - loss: 0.3603 - accuracy: 0.8532
Epoch 00005: saving model to checkpoints/weights.05.hdf5

Epoch 00005: saving model to checkpoints/weights.05.hdf5
27317/27317 [==============================] - 1s 39us/sample - loss: 0.3613 - accuracy: 0.8522
Epoch 6/100
   32/27317 [..............................] -

 3680/27317 [===>..........................] - ETA: 0s - loss: 0.3575 - accuracy: 0.8530
Epoch 00008: saving model to checkpoints/weights.08.hdf5
 5184/27317 [====>.........................] - ETA: 0s - loss: 0.3600 - accuracy: 0.8532
Epoch 00008: saving model to checkpoints/weights.08.hdf5

Epoch 00008: saving model to checkpoints/weights.08.hdf5
 6336/27317 [=====>........................] - ETA: 0s - loss: 0.3559 - accuracy: 0.8559
Epoch 00008: saving model to checkpoints/weights.08.hdf5
 7776/27317 [=======>......................] - ETA: 0s - loss: 0.3547 - accuracy: 0.8575
Epoch 00008: saving model to checkpoints/weights.08.hdf5
 9216/27317 [=========>....................] - ETA: 0s - loss: 0.3554 - accuracy: 0.8564
Epoch 00008: saving model to checkpoints/weights.08.hdf5

Epoch 00008: saving model to checkpoints/weights.08.hdf5
10432/27317 [==========>...................] - ETA: 0s - loss: 0.3563 - accuracy: 0.8567
Epoch 00008: saving model to checkpoints/weights.08.hdf5
11936/27

14912/27317 [===============>..............] - ETA: 0s - loss: 0.3631 - accuracy: 0.8521
Epoch 00010: saving model to checkpoints/weights.10.hdf5

Epoch 00010: saving model to checkpoints/weights.10.hdf5
16192/27317 [================>.............] - ETA: 0s - loss: 0.3625 - accuracy: 0.8525
Epoch 00010: saving model to checkpoints/weights.10.hdf5
17632/27317 [==================>...........] - ETA: 0s - loss: 0.3619 - accuracy: 0.8523
Epoch 00010: saving model to checkpoints/weights.10.hdf5
19168/27317 [====================>.........] - ETA: 0s - loss: 0.3609 - accuracy: 0.8528
Epoch 00010: saving model to checkpoints/weights.10.hdf5

Epoch 00010: saving model to checkpoints/weights.10.hdf5
20288/27317 [=====================>........] - ETA: 0s - loss: 0.3605 - accuracy: 0.8529
Epoch 00010: saving model to checkpoints/weights.10.hdf5
21760/27317 [======================>.......] - ETA: 0s - loss: 0.3611 - accuracy: 0.8529
Epoch 00010: saving model to checkpoints/weights.10.hdf5
23200/27

...

11136/27317 [===========>..................] - ETA: 0s - loss: 0.3535 - accuracy: 0.8539
Epoch 00089: saving model to checkpoints/weights.89.hdf5
12608/27317 [============>.................] - ETA: 0s - loss: 0.3559 - accuracy: 0.8523
Epoch 00089: saving model to checkpoints/weights.89.hdf5

Epoch 00089: saving model to checkpoints/weights.89.hdf5
13824/27317 [==============>...............] - ETA: 0s - loss: 0.3550 - accuracy: 0.8532
Epoch 00089: saving model to checkpoints/weights.89.hdf5
15296/27317 [===============>..............] - ETA: 0s - loss: 0.3547 - accuracy: 0.8541
Epoch 00089: saving model to checkpoints/weights.89.hdf5
16736/27317 [=================>............] - ETA: 0s - loss: 0.3528 - accuracy: 0.8550
Epoch 00089: saving model to checkpoints/weights.89.hdf5

Epoch 00089: saving model to checkpoints/weights.89.hdf5
17920/27317 [==================>...........] - ETA: 0s - loss: 0.3536 - accuracy: 0.8544
Epoch 00089: saving model to checkpoints/weights.89.hdf5
19264/27

22656/27317 [=======================>......] - ETA: 0s - loss: 0.3546 - accuracy: 0.8550
Epoch 00091: saving model to checkpoints/weights.91.hdf5
24128/27317 [=========================>....] - ETA: 0s - loss: 0.3540 - accuracy: 0.8551
Epoch 00091: saving model to checkpoints/weights.91.hdf5

Epoch 00091: saving model to checkpoints/weights.91.hdf5
25728/27317 [===========================>..] - ETA: 0s - loss: 0.3536 - accuracy: 0.8556
Epoch 00091: saving model to checkpoints/weights.91.hdf5
27317/27317 [==============================] - 1s 39us/sample - loss: 0.3533 - accuracy: 0.8557
Epoch 92/100
   32/27317 [..............................] - ETA: 1s - loss: 0.3028 - accuracy: 0.9062
Epoch 00092: saving model to checkpoints/weights.92.hdf5

Epoch 00092: saving model to checkpoints/weights.92.hdf5
 1472/27317 [>.............................] - ETA: 1s - loss: 0.3477 - accuracy: 0.8594
Epoch 00092: saving model to checkpoints/weights.92.hdf5
 2976/27317 [==>...........................] 

 6688/27317 [======>.......................] - ETA: 0s - loss: 0.3552 - accuracy: 0.8529
Epoch 00094: saving model to checkpoints/weights.94.hdf5
 7968/27317 [=======>......................] - ETA: 0s - loss: 0.3527 - accuracy: 0.8543
Epoch 00094: saving model to checkpoints/weights.94.hdf5

Epoch 00094: saving model to checkpoints/weights.94.hdf5
 9280/27317 [=========>....................] - ETA: 0s - loss: 0.3531 - accuracy: 0.8530
Epoch 00094: saving model to checkpoints/weights.94.hdf5
10848/27317 [==========>...................] - ETA: 0s - loss: 0.3521 - accuracy: 0.8549
Epoch 00094: saving model to checkpoints/weights.94.hdf5

Epoch 00094: saving model to checkpoints/weights.94.hdf5
12352/27317 [============>.................] - ETA: 0s - loss: 0.3544 - accuracy: 0.8535
Epoch 00094: saving model to checkpoints/weights.94.hdf5
13728/27317 [==============>...............] - ETA: 0s - loss: 0.3535 - accuracy: 0.8545
Epoch 00094: saving model to checkpoints/weights.94.hdf5
15168/27

Epoch 00096: saving model to checkpoints/weights.96.hdf5
18112/27317 [==================>...........] - ETA: 0s - loss: 0.3504 - accuracy: 0.8578
Epoch 00096: saving model to checkpoints/weights.96.hdf5
19200/27317 [====================>.........] - ETA: 0s - loss: 0.3521 - accuracy: 0.8562
Epoch 00096: saving model to checkpoints/weights.96.hdf5
20768/27317 [=====================>........] - ETA: 0s - loss: 0.3520 - accuracy: 0.8562
Epoch 00096: saving model to checkpoints/weights.96.hdf5
22176/27317 [=======================>......] - ETA: 0s - loss: 0.3524 - accuracy: 0.8554
Epoch 00096: saving model to checkpoints/weights.96.hdf5

Epoch 00096: saving model to checkpoints/weights.96.hdf5
23328/27317 [========================>.....] - ETA: 0s - loss: 0.3521 - accuracy: 0.8558
Epoch 00096: saving model to checkpoints/weights.96.hdf5
24832/27317 [==========================>...] - ETA: 0s - loss: 0.3528 - accuracy: 0.8554
Epoch 00096: saving model to checkpoints/weights.96.hdf5

Epoch 00

   32/27317 [..............................] - ETA: 0s - loss: 0.3068 - accuracy: 0.8438
Epoch 00099: saving model to checkpoints/weights.99.hdf5

Epoch 00099: saving model to checkpoints/weights.99.hdf5
 1664/27317 [>.............................] - ETA: 0s - loss: 0.3580 - accuracy: 0.8492
Epoch 00099: saving model to checkpoints/weights.99.hdf5
 2976/27317 [==>...........................] - ETA: 0s - loss: 0.3623 - accuracy: 0.8485
Epoch 00099: saving model to checkpoints/weights.99.hdf5
 4384/27317 [===>..........................] - ETA: 0s - loss: 0.3549 - accuracy: 0.8540
Epoch 00099: saving model to checkpoints/weights.99.hdf5

Epoch 00099: saving model to checkpoints/weights.99.hdf5
 5760/27317 [=====>........................] - ETA: 0s - loss: 0.3548 - accuracy: 0.8545
Epoch 00099: saving model to checkpoints/weights.99.hdf5
 7104/27317 [======>.......................] - ETA: 0s - loss: 0.3512 - accuracy: 0.8561
Epoch 00099: saving model to checkpoints/weights.99.hdf5
 8480/27

# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn_new = tf.keras.models.Sequential()

# First hidden layer
nn_new.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn_new.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn_new.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
nn_new.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Restore the model weights
nn_new.load_weights("checkpoints/weights.100.hdf5")

# Evaluate the model using the test data
model_loss, model_accuracy = nn_new.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

# Export our model to HDF5 file
nn_new.save("trained_attrition.h5")

# Import the model to a new object
nn_imported = tf.keras.models.load_model('trained_attrition.h5')

After running the code, we should see a file named “trained_attrition.h5,” which contains the complete model and configuration. Now that we have the model saved, we can create the model at any point. Let’s try importing the model into the notebook without providing any structure or context. To import the model, add and run the following code:

adding and running the following code:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_new.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")